# In this notebook we get the data from goodreads API that contains informations about books in the shelf of a given user 
>**The libiraries we used:**
  - **1-requests**
  - **2-BeautifulSoup**
  - **3-lxml**
  - **4-pandas**
  - **5-numpy**

In [1]:
import requests #to get the data from the APi

In [2]:
from bs4 import BeautifulSoup   #to make a better visualizing of the html code

In [3]:
from lxml import html     #to help search by path in the html code
import time               #to make a delay in the API requests

### The function idd make the requests given the user id 

In [4]:
def idd(idd):
    response=requests.get('https://www.goodreads.com/review/list?v=2&key=7q5vfUcnZX5crOGIHmQew&id='+idd)
    responsesoup=BeautifulSoup(response.content,'html.parser')
    return response

### Here we make the requests using users id from 11930000 to 11931970
**We made different ranges and store the data in different CSVs**

In [5]:
lst=[]
for i in range(11930000,11931970):
    books=idd(str(i))
    lst.append(books)
    time.sleep(1)

### Then we pick the good responses that we got from the requests

In [6]:
good_responses=[]
for i in lst:
    if str(i) == '<Response [200]>':
        good_responses.append(i)

### The function bs44 make two things:
**1- It makes the responses turn into a good-parsing html code**

**2- Then we get the body of the html code for each response (user) and store the body in a list**

In [7]:
def bs44(good_responses):
    lst=[]
    for i in good_responses:
        responsesoup=BeautifulSoup(i.content,'html.parser')
        html=list(responsesoup.children)[2]
        body=list(html.children)[3]
        books=list(body.children)
        lst.append(books)
    return lst

In [8]:
lst_users=bs44(good_responses)

### The function output has two parameters :
>  
   - **1-  lst_users : The books information for a specific user**
   - **2-  path : The path that you want to get information from in the html code**

In [9]:
def output(lst_users,path):
    i=1
    lst=[]
    while(i<len(lst_users)):
        path  = path
        byte_string = str(lst_users[i])

        # get filtered source code 
        source_code = html.fromstring(byte_string) 

        # jump to preferred html element 
        tree = source_code.xpath(path) 
        lst.append(tree[0].text_content())

        i+=2
    
    return lst

In [10]:
import pandas as pd  
import numpy as np  

### The (all_ ) list contains the 3 most important information that we want [book id, book title, user rating] for the first user 

In [11]:
all_=[output(lst_users[0],'//id[@type ="integer"]'),output(lst_users[0],'//title'),output(lst_users[0],'//rating')]

### Then we make a data frame using informaton of the first user

In [12]:
df=pd.DataFrame(np.array(all_[0]).reshape(-1,1),columns=['id'])
df['title']=all_[1]
df['rating']=all_[2]
df['user']=0

### The function user updates the data frame for each user

In [13]:
def user(i,df):
    lst=[output(lst_users[i],'//id[@type ="integer"]'),output(lst_users[i],'//title'),output(lst_users[i],'//rating')]
    df0=pd.DataFrame(np.array(lst[0]).reshape(-1,1),columns=['id'])
    df0['title']=lst[1]
    df0['rating']=lst[2]
    df0['user']=i
    df=pd.concat([df,df0])
    return df

### Here we update the data frame by the informations of all users we have 

In [14]:
for i in range(1,len(lst_users)):
    df=user(i,df)

### Then we save the data 

In [18]:
df.to_csv('Books1.csv',index=False)

### And that how it looks like 

In [435]:
df.head()

,id,title,rating,user
0,29056083,Harry Potter and the Cursed Child: Parts One a...,0,0
1,11125,Digital Fortress,4,0
2,976,Deception Point,4,0
3,6411961,"The Lost Symbol (Robert Langdon, #3)",4,0
4,968,"The Da Vinci Code (Robert Langdon, #2)",4,0
